In [1]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 9.1 MB/s eta 0:00:00


In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 64.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 91.0 MB/s eta 0:00:00


In [3]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import tensorflow_hub as hub
from sklearn.model_selection import train_test_split, GridSearchCV, KFold

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import StackingRegressor
from catboost import CatBoostRegressor
import xgboost as xgb

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error

from scipy.stats import norm
import numpy as np
import tensorflow as tf

In [4]:
url = "https://raw.githubusercontent.com/Tejanikhil/Shared-ASAG/main"
df = pd.read_csv(url + "/SemEvalData.csv")

In [5]:
import pandas as pd
import nltk
import numpy as np
from nltk.tokenize import word_tokenize, sent_tokenize
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.corpus import stopwords

# Download the stopwords corpus if not already downloaded
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [6]:
def extract_pos(text):
    tokens = word_tokenize(text)
    pos_tags = nltk.pos_tag(tokens)
    pos_words = [word for word, pos in pos_tags if pos.startswith('VB') or pos.startswith('JJ') or pos.startswith('NN')]
    return ' '.join(pos_words)

def process_dataset(text):
    lower_cased = text.lower()
    tokens = nltk.word_tokenize(lower_cased)
    filtered_tokens = [token for token in tokens if token not in stop_words]
    preprocessed = ' '.join(filtered_tokens)
    return preprocessed

In [7]:
df["concatenated"] = df['Question'] + " " + df['Student Answer']

In [8]:
from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(df.loc[:,["concatenated"]], df["Grade"], test_size=0.1, random_state=42)

In [9]:
train_X["concatenated"] = train_X["concatenated"].apply(extract_pos).apply(process_dataset)

test_X["concatenated"] = test_X["concatenated"].apply(extract_pos).apply(process_dataset)

In [10]:
sentences = list(train_X["concatenated"])

In [11]:
rm -rf ~/.cache/huggingface/transformers/bert-base-uncased

In [12]:
from transformers import BertTokenizer, BertModel
import torch

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [13]:
from transformers import BertTokenizer, BertModel
import torch

def get_bert_embeddings(sentence):

    input_ids = tokenizer.encode(sentence, add_special_tokens=True)
    input_ids = torch.tensor([input_ids])

    # Obtain BERT embeddings
    with torch.no_grad():
        outputs = model(input_ids)
        embeddings = outputs[0][:, 0, :]

    return np.array(embeddings)[0]

In [14]:
train_Embeddings_df = pd.DataFrame({"Q_Student_Answer_Embeddings":train_X["concatenated"]})
test_Embeddings_df = pd.DataFrame({"Q_Student_Answer_Embeddings":test_X["concatenated"]})

In [ ]:
# model = embed(sentences)

In [ ]:
# get_bert_embeddings(["Sentences"])

In [15]:
train_Embeddings_df["Q_Student_Answer_Embeddings"] = train_Embeddings_df["Q_Student_Answer_Embeddings"].apply(get_bert_embeddings)

test_Embeddings_df["Q_Student_Answer_Embeddings"] = test_Embeddings_df["Q_Student_Answer_Embeddings"].apply(get_bert_embeddings)


In [16]:
train_Embeddings_df

,Q_Student_Answer_Embeddings
199,"[-0.3265949, 0.10006174, -0.15291297, 0.035078..."
229,"[-0.5004935, -0.025693124, -0.18875223, 0.0301..."
521,"[-0.48320943, 0.023911811, -0.22422384, -0.217..."
651,"[-0.17587371, -0.28548425, -0.15400942, -0.110..."
450,"[-0.50214875, 0.071927406, -0.29693526, -0.136..."
...,...
1638,"[-0.50332636, -0.29232717, -0.069607854, -0.05..."
1095,"[-0.63312054, -0.033123787, -0.29795307, 0.121..."
1130,"[-0.5645942, -0.105580285, -0.11463924, 0.2052..."
1294,"[-0.7603619, -0.041815914, 0.31006297, 0.05276..."


In [17]:
train_X["QSA_Embeddings"] = train_Embeddings_df.apply(lambda x:x["Q_Student_Answer_Embeddings"], axis = 1)
test_X["QSA_Embeddings"] = test_Embeddings_df.apply(lambda x:x["Q_Student_Answer_Embeddings"], axis = 1)

In [18]:
train_df = pd.DataFrame(list(train_X["QSA_Embeddings"]))
test_df = pd.DataFrame(list(test_X["QSA_Embeddings"]))

In [19]:
train_df.to_csv("bert_train_embeddings_QSA.csv")
test_df.to_csv("bert_test_embeddings_QSA.csv")

In [20]:
from sklearn.linear_model import LinearRegression
LR_model = LinearRegression()
LR_model.fit(train_df, train_y)

LinearRegression()

In [21]:
preds = LR_model.predict(test_df)
y_preds = [round(y_val, 1) for y_val in preds]
actuals = test_y
y_true = [round(y_val, 1) for y_val in actuals]

In [22]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error
LR_mae = mean_absolute_error(y_true, np.round(y_preds, 1))
LR_mse = mean_squared_error(y_true, np.round(y_preds, 1))
LR_rmse = mean_squared_error(y_true, np.round(y_preds, 1), squared=False)
LR_mape = mean_absolute_percentage_error(y_true, y_preds, multioutput = "uniform_average")
LR_r2 = r2_score(y_true, y_preds)

In [23]:
print("Mean Absolute Error (MAE): ", np.round(LR_mae, 2))
print("Mean Squared Error (MSE): ", np.round(LR_mse, 2))
print("Root Mean Squared Error (RMSE): ", np.round(LR_rmse, 2))
print("Mean Absolute Percentage Error (MAPE): ", np.round(LR_mape, 2))
print("R2 Score: ", np.round(LR_r2, 2))

Mean Absolute Error (MAE):  1.34
Mean Squared Error (MSE):  2.99
Root Mean Squared Error (RMSE):  1.73
Mean Absolute Percentage Error (MAPE):  599254485850588.8
R2 Score:  -0.49


In [24]:
from sklearn import neighbors
rmse_val = []
mae_val = []
mse_val = []
r2_val = []
for K in range(2,20):
    knn_model = neighbors.KNeighborsRegressor(n_neighbors = K)
    knn_model.fit(train_df, train_y)  #fit the model
    pred=knn_model.predict(test_df) #make prediction on test set
    
    mae = mean_absolute_error(y_true, np.round(pred, 1))
    mae_val.append(mae)
    
    mse = mean_squared_error(y_true, np.round(pred, 1))
    mse_val.append(mse)
    
    rmse = mean_squared_error(y_true, np.round(pred, 1), squared=False)
    rmse_val.append(rmse)
    
    r2 = r2_score(y_true, pred)
    r2_val.append(r2)

In [25]:
y_true = test_y
knn_preds = knn_model.predict(test_df)

In [26]:
min_val = min(r2_val)
index = r2_val.index(min_val)
knn_r2 = min_val
knn_rmse = rmse_val[index]
knn_mae = mae_val[index]
knn_mse = mse_val[index]

In [27]:
print("Mean Absolute Error (MAE): ", np.round(knn_mae, 2))
print("Mean Squared Error (MSE): ", np.round(knn_mse, 2))
print("Root Mean Squared Error (RMSE): ", np.round(knn_rmse, 2))
print("R2 Score: ", np.round(knn_r2, 2))

Mean Absolute Error (MAE):  1.1
Mean Squared Error (MSE):  2.62
Root Mean Squared Error (RMSE):  1.62
R2 Score:  -0.3


In [28]:
ada_reg = AdaBoostRegressor(n_estimators = 100, learning_rate=0.5)

ada_reg.fit(train_df, train_y)

ada_reg_pred = ada_reg.predict(pd.DataFrame(test_df))
ada_reg_mae = mean_absolute_error(test_y, np.round(ada_reg_pred, 1))
ada_reg_mse = mean_squared_error(test_y, np.round(ada_reg_pred, 1))
ada_reg_mape = mean_absolute_percentage_error(test_y, np.round(ada_reg_pred,1), multioutput = "uniform_average")
ada_reg_rmse = mean_squared_error(test_y, np.round(ada_reg_pred, 1), squared=False)
ada_reg_r2 = r2_score(test_y, ada_reg_pred)

print("Mean Absolute Error (MAE): ", np.round(ada_reg_mae, 2))
print("Mean Squared Error (MSE): ", np.round(ada_reg_mse, 2))
print("Root Mean Squared Error (RMSE): ", np.round(ada_reg_rmse, 2))
print("Mean Absolute Percentage Error (MAPE): ", np.round(ada_reg_mape, 2))
print("R2 Score: ", np.round(ada_reg_r2, 2))

Mean Absolute Error (MAE):  1.36
Mean Squared Error (MSE):  2.37
Root Mean Squared Error (RMSE):  1.54
Mean Absolute Percentage Error (MAPE):  529402731707225.94
R2 Score:  -0.2


In [29]:
gb_reg = GradientBoostingRegressor(n_estimators = 50, max_depth=5)

gb_reg.fit(train_df, train_y)

gb_reg_pred = gb_reg.predict(pd.DataFrame(test_df))
gb_reg_mae = mean_absolute_error(test_y, np.round(gb_reg_pred, 1))
gb_reg_mse = mean_squared_error(test_y, np.round(gb_reg_pred, 1))
gb_reg_mape = mean_absolute_percentage_error(test_y, np.round(gb_reg_pred,1), multioutput = "uniform_average")
gb_reg_rmse = mean_squared_error(test_y, np.round(gb_reg_pred, 1), squared=False)
gb_reg_r2 = r2_score(test_y, gb_reg_pred)

print("Mean Absolute Error (MAE): ", np.round(gb_reg_mae, 2))
print("Mean Squared Error (MSE): ", np.round(gb_reg_mse, 2))
print("Root Mean Squared Error (RMSE): ", np.round(gb_reg_rmse, 2))
print("Mean Absolute Percentage Error (MAPE): ", np.round(gb_reg_mape, 2))
print("R2 Score: ", np.round(gb_reg_r2, 2))

Mean Absolute Error (MAE):  1.1
Mean Squared Error (MSE):  1.82
Root Mean Squared Error (RMSE):  1.35
Mean Absolute Percentage Error (MAPE):  573519626016161.4
R2 Score:  0.08


In [30]:
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', seed = 123)

xg_reg.fit(train_df, train_y)

# print("Best parameters: ", xg_boost_grid_search.best_params_)
# print("Best score: ", xg_boost_grid_search.best_score_)
xg_reg_pred = xg_reg.predict(pd.DataFrame(test_df))
xg_reg_mae = mean_absolute_error(test_y, np.round(xg_reg_pred, 1))
xg_reg_mse = mean_squared_error(test_y, np.round(xg_reg_pred, 1))
xg_reg_mape = mean_absolute_percentage_error(test_y, np.round(xg_reg_pred,1), multioutput = "uniform_average")
xg_reg_rmse = mean_squared_error(test_y, np.round(xg_reg_pred, 1), squared=False)
xg_reg_r2 = r2_score(test_y, xg_reg_pred)

print("Mean Absolute Error (MAE): ", np.round(xg_reg_mae, 2))
print("Mean Squared Error (MSE): ", np.round(xg_reg_mse, 2))
print("Root Mean Squared Error (RMSE): ", np.round(xg_reg_rmse, 2))
print("Mean Absolute Percentage Error (MAPE): ", np.round(xg_reg_mape, 2))
print("R2 Score: ", np.round(xg_reg_r2, 2))

Mean Absolute Error (MAE):  1.18
Mean Squared Error (MSE):  2.31
Root Mean Squared Error (RMSE):  1.52
Mean Absolute Percentage Error (MAPE):  597416277100168.1
R2 Score:  -0.17
